In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import os
from torch.utils.data import random_split
import glob

### Import dataset and put into dataloader

In [ ]:
'''
##### import CIFAR-10 dataset into trainset, testset# color dataset
transform_color = transforms.Compose([
        transforms.ToTensor(),
         transforms.Normalize((0.5), (0.5))
    ])

batchSize = 20

trainset_color = torchvision.datasets.CIFAR10(root='../input/cifar10-python', train=True,
                                         download=False, transform=transform_color)
trainloader_color = torch.utils.data.DataLoader(trainset_color, batch_size=batchSize,
                                          shuffle=False, num_workers=2)

testset_color = torchvision.datasets.CIFAR10(root='../input/cifar10-python', train=False,
                                       download=False, transform=transform_color)
testloader_color = torch.utils.data.DataLoader(testset_color, batch_size=batchSize,
                                         shuffle=False, num_workers=2)

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print(device)
'''

**Splitting the training dataset for training and validation**

In [ ]:

##### import CIFAR-10 dataset into trainset, testset# color dataset
transform_color = transforms.Compose([
        transforms.ToTensor(),
         transforms.Normalize((0.5), (0.5))
    ])

batchSize = 16

trainset_color = torchvision.datasets.CIFAR10(root='../input/cifar10-python', train=True,
                                        download=False, transform=transform_color)


testset_color = torchvision.datasets.CIFAR10(root='../input/cifar10-python', train=False,
                                       download=False, transform=transform_color)

torch.manual_seed(43)
val_size = 5000
train_size = len(trainset_color) - val_size

trainset_color, valset_color = random_split(trainset_color, [train_size, val_size])
print(len(trainset_color), len(valset_color))

valloader_color = torch.utils.data.DataLoader(valset_color, batch_size=batchSize,
                                          shuffle=False, num_workers=2)

trainloader_color = torch.utils.data.DataLoader(trainset_color, batch_size=batchSize,
                                          shuffle=False, num_workers=2)
testloader_color = torch.utils.data.DataLoader(testset_color, batch_size=batchSize,
                                         shuffle=False, num_workers=2)
# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print(device)


### Pre-processing: Convert all rgb images to gray images

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# get some random training images
dataiter = iter(trainloader_color)

images_color, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images_color))

# show dataiter shape
print(images_color.shape)

for i, data_color in enumerate(trainloader_color, 0):
    if (i==0):
        images_color, labels_color = data_color
        imshow(torchvision.utils.make_grid(images_color))
        break
    

In [ ]:
# function to convert a rgb_img to gray_img
def rgb2gray(img):
    rgb_img = img.transpose(1,2,0)
    gray_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2GRAY)
    return gray_img

# function to convert a batch of rgb images to gray images
def rgb2gray_batch(images):
    rgb_images = np.array(images)    
    
    gray_images = []
    for i in range(rgb_images.shape[0]):
        img = rgb_images[i]
        gray_img = rgb2gray(img)
        gray_images.append(gray_img)
    
    gray_images = np.array(gray_images)
    gray_images = torch.from_numpy(gray_images)
    return gray_images
        
images_gray = rgb2gray_batch(images_color)

# show images
images_gray = images_gray.reshape(batchSize,1,32,32)

imshow(torchvision.utils.make_grid(images_color))
imshow(torchvision.utils.make_grid(images_gray))
print("images_gray shape: ", images_gray.shape)

### Define the MLP network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class MLP_net(nn.Module):
    def __init__(self):
        # Input layer is 1024, since CIFAR-10 gray_image is 32x32
        # Output layer is 3072, since CIFAR-10 gray_image is 32x32x3
        super(MLP_net, self).__init__()
        
        self.layer1 = nn.Linear(1024,512)
        self.bn1 = nn.BatchNorm1d(512)
        self.layer2 = nn.Linear(512,256)
        self.bn2 = nn.BatchNorm1d(256)
        self.layer3 = nn.Linear(256,128)
        self.bn3 = nn.BatchNorm1d(128)
        self.layer4 = nn.Linear(128,256)
        self.bn4 = nn.BatchNorm1d(256)
        self.layer5 = nn.Linear(256,512)
        self.bn5 = nn.BatchNorm1d(512)
        self.layer6 = nn.Linear(512,1024)
        self.bn6 = nn.BatchNorm1d(1024)
        self.layer7 = nn.Linear(1024,3072)

        
    def forward(self, x):
        # convert tensor (4, 1, 32, 32) --> (4, 1*32*32)
        x = x.view(x.size(0), -1)
        
        x = self.layer1(x)
        x = self.bn1(x)
        x = self.layer2(x)
        x = self.bn2(x)
        x = self.layer3(x)
        x = self.bn3(x)
        x = self.layer4(x)
        x = self.bn4(x)
        x = self.layer5(x)
        x = self.bn5(x)
        x = self.layer6(x)
        x = self.bn6(x)
        x = self.layer7(x)
        return x

'''
class MLP_net(nn.Module):
    def __init__(self):
        # Input layer is 1024, since CIFAR-10 gray_image is 32x32
        # Output layer is 3072, since CIFAR-10 gray_image is 32x32x3
        super(MLP_net, self).__init__()
        
        self.layer1 = nn.Linear(1024,512)
        self.bn1 = nn.BatchNorm1d(512)
        self.layer2 = nn.Linear(512,256)
        self.bn2 = nn.BatchNorm1d(256)
        self.layer3 = nn.Linear(256,128)
        self.bn3 = nn.BatchNorm1d(128)
        self.layer4 = nn.Linear(128,256)
        self.bn4 = nn.BatchNorm1d(256)
        self.layer5 = nn.Linear(256,512)
        self.bn5 = nn.BatchNorm1d(512)
        self.layer6 = nn.Linear(512,1024)
        self.bn6 = nn.BatchNorm1d(1024)
        self.layer7 = nn.Linear(1024,3072)
        
        /*

        
    def forward(self, x):
        # convert tensor (4, 1, 32, 32) --> (4, 1*32*32)
        x = x.view(x.size(0), -1)
        
        x = self.layer1(x)
        x = self.bn1(x)
        x = self.layer2(x)
        x = self.bn2(x)
        x = self.layer3(x)
        x = self.bn3(x)
        x = self.layer4(x)
        x = self.bn4(x)
        x = self.layer5(x)
        x = self.bn5(x)
        x = self.layer6(x)
        x = self.bn6(x)
        x = self.layer7(x)
        return x

'''

mlp_net = MLP_net().to(device)

print("is cude: ", next(mlp_net.parameters()).is_cuda)

if os.path.exists("./saved_model/mlp.pt"):
    print("Continue training from the saved model")
    mlp_net.load_state_dict(torch.load("./saved_model/mlp.pt"))

### Define the loss function and optimizer

In [ ]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(mlp_net.parameters(), lr=0.001, momentum=0.9)

### Tensorboard to save all training output logs

In [ ]:

from torch.utils.tensorboard import SummaryWriter

# Writer will output to ./runs/ directory by default
writer = SummaryWriter("./runs_mlp/")

### Train the network on 1 batch

In [ ]:
dataiter = iter(trainloader_color)

images_color, labels = dataiter.next()

history_loss=[]
history_val_loss=[]
for epoch in range(8000):  # loop over the dataset multiple times
    
    running_loss = 0.0
    num_training = 0
    
    # get the inputs; data is a list of [inputs, labels]
    images_color, labels_color = data_color
    images_gray = rgb2gray_batch(images_color)
    # put data in gpu/cpu
    images_color = images_color.to(device)
    images_gray = images_gray.to(device)
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    # input: images_gray
    images_gray = images_gray.view(images_gray.shape[0],-1)
    # print("images_gray: ", images_gray.shape)
    outputs = mlp_net(images_gray)
    # change the shape of images_color: [batchSize,3,32,32] -> [batchSize,3072]
    images_color = images_color.reshape(images_color.shape[0], -1) 
    loss = criterion(outputs, images_color)
    loss.backward()
    optimizer.step()
    # print output statistics
    running_loss += loss.item()
    j = 1          # print every 200 mini-batches
    if i % j == (j-1):     # print every 200 mini-batches
        print("trainingloss:",'[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / j))
        writer.add_scalar('Loss/train', running_loss/j, len(trainset_color)*epoch + i)
        history_loss.append(running_loss / j)
        running_loss = 0.0
            
    
    running_val_loss = 0.0
    # get the inputs; data is a list of [inputs, labels]
    images_val_color, labels_val_color = data_val_color
    images_val_gray = rgb2gray_batch(images_val_color)
    # put data in gpu/cpu
    images_val_color = images_val_color.to(device)
    images_val_gray = images_val_gray.to(device)

    # forward + backward + optimize
    # input: images_val_gray
    images_val_gray = images_val_gray.view(images_val_gray.shape[0],-1)
    # print("images_val_gray: ", images_val_gray.shape)
    outputs_val = mlp_net(images_val_gray)
    # change the shape of images_val_color: [batchSize,3,32,32] -> [batchSize,3072]
    images_val_color = images_val_color.reshape(images_val_color.shape[0], -1) 

    loss_val = criterion(outputs_val, images_val_color)

    # print output statistics
    running_val_loss += loss_val.item()


    j = 1      # print every 200 mini-batches #TODO mettre le meme j que au dessus
    if i % j == (j-1):     # print every 200 mini-batches
        print("validation",'[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_val_loss / j))
        writer.add_scalar('Loss/val', running_val_loss/j, len(valset_color)*epoch + i)
        history_val_loss.append(running_val_loss / j)

        running_val_loss = 0.0
        
writer.flush()
writer.close()
print('Finished Training')

In [ ]:
'''
history_loss=[]
history_val_loss=[]
for epoch in range(8000):  # loop over the dataset multiple times
    
    running_loss = 0.0
    num_training = 0
    
    for i, data_color in enumerate(, 0):
        # get the inputs; data is a list of [inputs, labels]
        images_color, labels_color = data_color
        images_gray = rgb2gray_batch(images_color)
        # put data in gpu/cpu
        images_color = images_color.to(device)
        images_gray = images_gray.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        # input: images_gray
        images_gray = images_gray.view(images_gray.shape[0],-1)
        # print("images_gray: ", images_gray.shape)
        outputs = mlp_net(images_gray)
        # change the shape of images_color: [batchSize,3,32,32] -> [batchSize,3072]
        images_color = images_color.reshape(images_color.shape[0], -1) 
        loss = criterion(outputs, images_color)
        loss.backward()
        optimizer.step()
        # print output statistics
        running_loss += loss.item()
        j = 1            # print every 200 mini-batches
        if i % j == (j-1):     # print every 200 mini-batches
            print("trainingloss:",'[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / j))
            writer.add_scalar('Loss/train', running_loss/j, len(trainset_color)*epoch + i)
            history_loss.append(running_loss / j)
            running_loss = 0.0
            
    
    running_val_loss = 0.0
    for i, data_val_color in enumerate(valloader_color, 0):
        # get the inputs; data is a list of [inputs, labels]
        images_val_color, labels_val_color = data_val_color
        images_val_gray = rgb2gray_batch(images_val_color)
        # put data in gpu/cpu
        images_val_color = images_val_color.to(device)
        images_val_gray = images_val_gray.to(device)
        
        # forward + backward + optimize
        # input: images_val_gray
        images_val_gray = images_val_gray.view(images_val_gray.shape[0],-1)
        # print("images_val_gray: ", images_val_gray.shape)
        outputs_val = mlp_net(images_val_gray)
        # change the shape of images_val_color: [batchSize,3,32,32] -> [batchSize,3072]
        images_val_color = images_val_color.reshape(images_val_color.shape[0], -1) 
        
        loss_val = criterion(outputs_val, images_val_color)
        
        # print output statistics
        running_val_loss += loss_val.item()
        
        
        j = 1         # print every 200 mini-batches #TODO mettre le meme j que au dessus
        if i % j == (j-1):     # print every 200 mini-batches
            print("validation",'[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_val_loss / j))
            writer.add_scalar('Loss/val', running_val_loss/j, len(valset_color)*epoch + i)
            history_val_loss.append(running_val_loss / j)
            
            running_val_loss = 0.0
        
writer.flush()
writer.close()
print('Finished Training')
'''

### Loss graphic evolution displayed (on training set and validation set)

In [ ]:
# Defining Figure
f = plt.figure(figsize=(10,7))
f.add_subplot()


#Adding Subplot
plt.plot(range(len(history_loss)), history_loss, label = "loss") # Loss curve for training set
plt.plot(range(len(history_val_loss)), history_val_loss, label = "val_loss") # Loss curve for validation set

plt.title("Loss Curve",fontsize=18)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Loss",fontsize=15)
plt.grid(alpha=0.3)
plt.legend()
plt.savefig("Loss_curve.png")
plt.show()

### Inference on train dataset

In [ ]:

# get some random training images
#dataiter = iter(trainloader_color)
#images_color, labels_color = dataiter.next()

for i, data_color in enumerate(trainloader_color, 0):
    if (i==0):
        images_color, labels_color = data_color
        imshow(torchvision.utils.make_grid(images_color))
        break

print("images_color: ", images_color.shape)
# show images
images_color_show = images_color.reshape(batchSize,3,32,32)
imshow(torchvision.utils.make_grid(images_color_show.detach()))

images_gray = rgb2gray_batch(images_color)
#print("images_gray: ", images_gray.shape)
# show images
images_gray = images_gray.reshape(batchSize,1,32,32)
imshow(torchvision.utils.make_grid(images_gray.detach()))

# run inference on the network
# oututs [4,3072]
images_gray = images_gray.to(device)
outputs = mlp_net(images_gray)
#print(outputs.shape)

images_color = outputs.reshape(batchSize,3,32,32)
#print(images_color.shape)

images_color_1=images_color.reshape(images_color.shape[0], -1)

loss_val = criterion(outputs, images_color_1)
# print output statistics
running_val_loss=0
running_val_loss += loss_val.item()
#print(loss_val.item())

images_gray = images_gray.to("cpu")
images_color = images_color.to("cpu")
imshow(torchvision.utils.make_grid(images_color.detach()))

images_gray = images_gray.to("cpu")
images_gray = torch.cat((images_gray, images_gray, images_gray), 1)

final_result_display = torch.cat((images_color_show, images_gray, images_color), 0)
imshow(torchvision.utils.make_grid(final_result_display.detach()))
'''
imshow(torchvision.utils.make_grid(images_color_show[1].detach()))
imshow(torchvision.utils.make_grid(images_gray[1].detach()))
imshow(torchvision.utils.make_grid(images_color[1].detach()))
'''

In [ ]:
imshow(torchvision.utils.make_grid(images_color_show[11].detach()))
imshow(torchvision.utils.make_grid(images_gray[11].detach()))
imshow(torchvision.utils.make_grid(images_color[11].detach()))

### Save the model

In [ ]:
if not os.path.exists('./saved_model'):
    os.makedirs('./saved_model')
    
torch.save(mlp_net.state_dict(), './saved_model/mlp.pt')

### Inference on validation dataset

In [ ]:
# get some random training images
#dataiter = iter(valloader_color)
#images_color, labels_color = dataiter.next()
for i, data_color in enumerate(valloader_color, 0):
    if (i==4):
        images_color, labels_color = data_color
        imshow(torchvision.utils.make_grid(images_color))
        break
#print("images_color: ", images_color.shape)
# show images
images_color_show = images_color.reshape(batchSize,3,32,32)
imshow(torchvision.utils.make_grid(images_color_show.detach()))

images_gray = rgb2gray_batch(images_color)
#print("images_gray: ", images_gray.shape)
# show images
images_gray = images_gray.reshape(batchSize,1,32,32)
imshow(torchvision.utils.make_grid(images_gray.detach()))

# run inference on the network
# oututs [4,3072]
images_gray = images_gray.to(device)
outputs = mlp_net(images_gray)
#print(outputs.shape)

images_color = outputs.reshape(batchSize,3,32,32)
#print(images_color.shape)

images_gray = images_gray.to("cpu")
images_color = images_color.to("cpu")
imshow(torchvision.utils.make_grid(images_color.detach()))


images_gray = images_gray.to("cpu")
images_gray = torch.cat((images_gray, images_gray, images_gray), 1)

final_result_display = torch.cat((images_color_show, images_gray, images_color), 0)
imshow(torchvision.utils.make_grid(final_result_display.detach()))
'''
imshow(torchvision.utils.make_grid(images_color_show[5].detach()))
imshow(torchvision.utils.make_grid(images_gray[5].detach()))
imshow(torchvision.utils.make_grid(images_color[5].detach()))
'''

### Inference on test dataset

In [ ]:
# get some random training images
dataiter = iter(testloader_color)
images_color, labels_color = dataiter.next()
#print("images_color: ", images_color.shape)
# show images
images_color_show = images_color.reshape(batchSize,3,32,32)
imshow(torchvision.utils.make_grid(images_color_show.detach()))

images_gray = rgb2gray_batch(images_color)
#print("images_gray: ", images_gray.shape)
# show images
images_gray = images_gray.reshape(batchSize,1,32,32)
imshow(torchvision.utils.make_grid(images_gray.detach()))

# run inference on the network
# oututs [batchSize,3072]
images_gray = images_gray.to(device)
outputs = mlp_net(images_gray)
#print(outputs.shape)

images_color = outputs.reshape(batchSize,3,32,32)
#print(images_color.shape)

images_color = images_color.to("cpu")
imshow(torchvision.utils.make_grid(images_color.detach()))

images_gray = images_gray.to("cpu")
images_gray = torch.cat((images_gray, images_gray, images_gray), 1)

final_result_display = torch.cat((images_color_show, images_gray, images_color), 0)
imshow(torchvision.utils.make_grid(final_result_display.detach()))
